In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout, Activation, Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.applications.inception_v3 import InceptionV3

from dataset import getDataset
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

import os
import cv2
import time
import random

In [2]:
IMG_SIZE = 224

X, y = getDataset(IMG_SIZE)

print(X.shape, y.shape)

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)


X_aug, y_aug = [], []

cnt = 0
for _X, _y in datagen.flow(X, y):    
    for i in range(_X.shape[0]):
        cnt += 1
        X_aug.append(_X[i])
        y_aug.append(_y[i])
    
    if cnt >= 1000:
        break

X = np.array(X_aug).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y_aug).reshape(-1, 1)

X.shape, y.shape

(92, 224, 224, 3) (92, 1)


/home/roshan_/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/roshan_/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


((1012, 224, 224, 3), (1012, 1))

In [3]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np

model = InceptionV3(weights='imagenet', include_top=False)

X_features = []
for img in X:
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    
    X_features.append(features)

print(len(X_features), X_features[0].shape)

h, w, c = X_features[0].shape[1:]
X = np.array(X_features).reshape(-1, h, w, c)
X.shape, y.shape

1012 (1, 5, 5, 2048)


((1012, 5, 5, 2048), (1012, 1))

In [4]:
model = Sequential()

model.add(Flatten(input_shape=(X.shape[1:])))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 51200)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                3276864   
_________________________________________________________________
activation_94 (Activation)   (None, 64)                0         
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                1040      
_________________________________________________________________
activation_95 (Activation)   (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0

In [5]:
history = model.fit(X, y, validation_split=0.3, batch_size=32, epochs=50, shuffle=True)

Epoch 1/50
23/23 [==============================] - 1s 30ms/step - loss: 2.3519 - accuracy: 0.6215 - val_loss: 0.1971 - val_accuracy: 0.9243
Epoch 2/50
23/23 [==============================] - 0s 18ms/step - loss: 0.8484 - accuracy: 0.8305 - val_loss: 0.2545 - val_accuracy: 0.9276
Epoch 3/50
23/23 [==============================] - 0s 18ms/step - loss: 0.3128 - accuracy: 0.9195 - val_loss: 0.0629 - val_accuracy: 0.9868
Epoch 4/50
23/23 [==============================] - 0s 17ms/step - loss: 0.1164 - accuracy: 0.9548 - val_loss: 0.0650 - val_accuracy: 0.9803
Epoch 5/50
23/23 [==============================] - 0s 17ms/step - loss: 0.1585 - accuracy: 0.9605 - val_loss: 0.0542 - val_accuracy: 0.9836
Epoch 6/50
23/23 [==============================] - 0s 17ms/step - loss: 0.0851 - accuracy: 0.9788 - val_loss: 0.0480 - val_accuracy: 0.9868
Epoch 7/50
23/23 [==============================] - 0s 17ms/step - loss: 0.1101 - accuracy: 0.9661 - val_loss: 0.3345 - val_accuracy: 0.9276
Epoch 8/50
23